<a href="https://colab.research.google.com/github/amzker/ML-Algorithms-From-Scratch/blob/main/Regression/chapter_1/Gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATAPREP

WE WILL USE SAME DATASET AS BEFORE BUT YOU SHOULD USE DIFFERENT DATASET


In [2]:
import numpy as np
!pip install snoop
import snoop
import pandas as pd
snoop.install()


# Snoop is good debugger, you will need it to check your calculations, and it saves time a lot 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

# i am using standerd scaler. what it does is that x - meanx/std thus makes our dataset to have variance of 1
# which is helpful in case of gradeint calculations. YOU SHOULD DO IT MANUALLY 
# I WAS IN HURRY FOR TESTING THE CODE SO DIDNT BOTHER MUCH: FORMULA FOR SCALE = (X - X.mean())/X.STD(), TRY IT USING NP.MEAN AND NP.STD ALSO 

data = pd.read_csv("https://raw.githubusercontent.com/amzker/ML-Algorithms-From-Scratch/main/Regression/chapter_0/datasets/MLinear.csv")

# DATA FOR SIMPLE LINEAR REGRESSION
sy = data["Number of weekly drivers"]
sx = data["Population of city"]
sy = np.asarray(sy).reshape(-1,1)
sx = np.asarray(sx).reshape(-1,1)
sx = sc.fit_transform(sx)
# DATA FOR MULTIPLE LINEAR REGRESSION
y = data["Number of weekly drivers"]
X = data.drop(columns=["Number of weekly drivers","City "])
y = np.asarray(y).flatten()
X = np.asarray(X)
X = sc.fit_transform(X)
# DATA FOR MULTI TARGET REGRESSION
yy = data[["Number of weekly drivers","Population of city"]]
XX = data.drop(columns=["Number of weekly drivers","City ","Population of city"])
yy = np.asarray(yy)
XX = np.asarray(XX)
XX = sc.fit_transform(XX)

# Simple Linear Regression

$$ dm = -2 \sum x_i(y_i - mx_i- c)/N$$
$$$$
$$N = len(x)$$





In [4]:
class LinGD:
  def __init__(self):
    self.b = None
    self.c = None
    self.mse = None
    self.count = None
  def fit(self,X,y,lr,ep):
    self.b = 0
    self.c = 0
    self.count = 0
    for i in range(ep):
      pred = np.dot(self.b,X) + self.c
      error = y - pred
      db = ((-2)/len(X)) * np.sum(X*error)
      self.b -= lr* db
      self.c -= -2 *lr* np.mean(error)
      if self.mse == np.mean(error*error):
        self.count += 1
      if self.count >4:
        print("Converged at: EPOCH: ",i,self.__dict__)
        break
      self.mse = np.mean(error*error)
      #print("EPOCH:",i,self.__dict__)
lng = LinGD()
lng.fit(sx,sy,0.01,1000)

Converged at: EPOCH:  954 {'b': 18761.727554062003, 'c': 160026.073405587, 'mse': 84929257.8704093, 'count': 5}


In [38]:
from sklearn.linear_model import LinearRegression
skl = LinearRegression()
skl.fit(sx,sy)
skl.__dict__

{'fit_intercept': True,
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'n_features_in_': 1,
 'coef_': array([[18761.72763244]]),
 'rank_': 1,
 'singular_': array([5.19615242]),
 'intercept_': array([160026.07407407])}

# Multi Linear Regression 

$$ db = -2 \sum X^T_i(Y_i - βX_i)/N $$


In [74]:
class MDS:
  def __init__(self):
    self.beta = 0
  

  def fit(self,X,y,lr,ep):
    X = np.insert(np.array(X), 0, 1, axis=1) #array,posotion,value to add,axis
    beta = np.ones(X.shape[1])
    #print(self.beta)
    for i in range(ep):
        pred = X.dot(beta)
        error = y- pred
        db = (-2)/len(X) * np.dot(X.T,error)
        beta = beta - lr*db
        self.beta = beta[1:]
        self.c = beta[0]


{'beta': array([-18671.59902574,   3885.28510309,  -4068.21612221,   6694.34632891]),
 'c': 160026.07407407396}

In [54]:
ms = MDS()
ms.fit(X,y,0.05,1000)
ms.__dict__

{'beta': array([ 1.02131280e+142, -1.06283629e+142,  1.06606418e+142,
         1.03788534e+142]),
 'c': -5.547962430630156e+126}

In [343]:
from sklearn.linear_model import LinearRegression
mm = LinearRegression()
mm.fit(X,y)
mm.__dict__

{'fit_intercept': True,
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'n_features_in_': 4,
 'coef_': array([-18693.97136508,   3655.55168543,  -4363.17091585,   6784.06807352]),
 'rank_': 4,
 'singular_': array([10.06339977,  2.07311856,  1.34571229,  0.78690713]),
 'intercept_': 160026.07407407407}

# Multi Target Linear Regression

$$ db = -2 \sum X^T_i(Y_i - βX_i- C)/N $$

$$$$
$$ dc = -2 \sum (Y_i - βX_i- C)/N$$

note:we will need separate Cs because they are going to be equal to number of ys

In [100]:
class MDSs:
  def __init__(self):
    self.beta = None
    self.c = None

  def fit(self, X, y, lr=0.01, epochs=100):
    #X = np.insert(np.array(X), 0, 1, axis=1) #? guess why i commented this one here?
    n_samples, n_features = X.shape
    n_outputs = y.shape[1]
    self.beta = np.zeros((n_features, n_outputs))
    self.c = np.zeros(n_outputs)
    
    for i in range(epochs):
      pred = X.dot(self.beta) + self.c
      error = y - pred  
      db = -2/len(X) * X.T.dot(error)
      dc =  (-2) * np.mean(error, axis=0)
      self.beta = self.beta - (lr * db)
      self.c = self.c - (lr * dc)
    return self
    
  def predict(self, X):
    return X.dot(self.beta) + self.c
news = MDSs()
news.fit(XX,yy,0.02,10000)
news.__dict__

{'beta': array([[-19513.88790403, -14923.50603333],
        [ -7244.483981  , -52443.50188761],
        [  6853.63980813,   1266.29334781]]),
 'c': array([ 160026.07407407, 1680111.11111111])}

In [97]:
from sklearn.linear_model import LinearRegression
mms = LinearRegression()
mms.fit(XX,yy)
mms.__dict__

{'fit_intercept': True,
 'copy_X': True,
 'n_jobs': None,
 'positive': False,
 'n_features_in_': 3,
 'coef_': array([[-19513.8878795 ,  -7244.48407231,   6853.63987696],
        [-14923.50586442, -52443.50251658,   1266.29382194]]),
 'rank_': 3,
 'singular_': array([8.68975569, 2.07256592, 1.09206985]),
 'intercept_': array([ 160026.07407407, 1680111.11111111])}

In [54]:
cmd = pd.DataFrame()
cmd[["actual1","actual2"]] =  data[["Number of weekly drivers","Population of city"]]
cmd[["my preds1","mypred2"]] = news.predict(XX)
cmd[["SKlearn1","sklearn2"]] = mms.predict(XX)
cmd

,actual1,actual2,my preds1,mypred2,SKlearn1,sklearn2
0,192000,1800000,187309.848974,1.776837e+06,187752.302897,1.780040e+06
1,190400,1790000,186642.335239,1.772063e+06,186888.001982,1.773784e+06
2,191200,1780000,187448.909288,1.767845e+06,187996.049864,1.770940e+06
3,177600,1778000,180122.695902,1.759933e+06,180582.323136,1.763871e+06
4,176800,1750000,180039.256685,1.759336e+06,180474.285521,1.763089e+06
5,178400,1740000,181129.524072,1.757147e+06,181949.661292,1.762904e+06
6,180800,1725000,178996.258902,1.736897e+06,179219.341756,1.737707e+06
7,175200,1725000,174749.077690,1.728764e+06,174756.215091,1.728698e+06
8,174400,1720000,174415.320822,1.726377e+06,174324.064634,1.725569e+06
9,173920,1705000,174317.972545,1.720687e+06,174229.862889,1.719455e+06
